In [4]:
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Times",
    "font.size": 14,
    "axes.titlesize": 16,
    "axes.labelsize": 16,
    "axes.linewidth": 1.5,
    "lines.linewidth": 2,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "xtick.major.width": 1.2,
    "ytick.major.width": 1.2,
    "legend.fontsize": 14,
    "figure.dpi": 300,
    "savefig.dpi": 300,
    "figure.figsize": (8, 5),
})

import seaborn as sns
sns.set_style("white")
sns.set_context("paper")

In [5]:
cs = np.linspace(0, 1, 257)

sparse_palette = sns.color_palette("Reds", n_colors=4)
spectral_palette = sns.color_palette("Blues", n_colors=4)

color_map = {
    "weighted_rand_sparse_0": sparse_palette[0],
    "weighted_rand_sparse_1": sparse_palette[1],
    "weighted_rand_sparse_2": sparse_palette[2],
    "exact_sparse": sparse_palette[3],
    "rand_svd_0": spectral_palette[0],
    "rand_svd_1": spectral_palette[1],
    "rand_svd_2": spectral_palette[2],
    "low_rank": spectral_palette[3],
    "quantize": "black",
}

label_map = {
    "weighted_rand_sparse_0": r"$\mathrm{RandSparse_0}$",
    "weighted_rand_sparse_1": r"$\mathrm{RandSparse_1}$",
    "weighted_rand_sparse_2": r"$\mathrm{RandSparse_2}$",
    "exact_sparse": r"$\mathrm{Sparse}$",
    "rand_svd_0": r"$\mathrm{RandSVD_0}$",
    "rand_svd_1": r"$\mathrm{RandSVD_1}$",
    "rand_svd_2": r"$\mathrm{RandSVD_2}$",
    "low_rank": r"$\mathrm{SVD}$",
    "quantize": r"$\mathrm{Quant}$",
}

def pretty_graph():
    sns.despine()
    plt.tight_layout()
    plt.gca().xaxis.set_ticks_position("bottom")
    plt.gca().yaxis.set_ticks_position("left")
    plt.legend(frameon=False)


In [24]:
a = np.load("data/distilbert_ag_news/aA_ab_acc_low_rank_full.npy")
np.save("data/distilbert_ag_news/aA_ab_acc_low_rank.npy", a[744:])

In [ ]:
# metric = "l2"
metric = "acc"
# metric = "loss"

ylabel = {
    "l2": r"$\ell_2$ distance",
    "acc": "accuracy",
    "loss": "negative log-likelihood loss",
}

experiment = "alexnet_cifar10"
# experiment = "distilbert_ag_news"

if experiment == "alexnet_cifar10":
    cs = np.linspace(0, 1, 257)[224:]
elif experiment == "distilbert_ag_news":
    cs = np.linspace(0, 1, 769)


for method, color in color_map.items():
    sns.lineplot(
        x=cs,
        y=np.load(f"data/{experiment}/aA_ab_{metric}_{method}.npy"),
        color=color,
        label=label_map[method],
        linewidth=1
    )

pretty_graph()
plt.xlabel("compression rate")
plt.ylabel(ylabel[metric])
plt.savefig(f"diagrams/{experiment}/aA_ab_{metric}.pdf", bbox_inches="tight")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/poset/Desktop/MIT/ml_approx/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
  File "/var/folders/w0/14p5k55x66q23_9t9vhg92wm0000gn/T/ipykernel_79653/2186714340.py", line 21, in <module>
    sns.lineplot(
  File "/Users/poset/Desktop/MIT/ml_approx/.venv/lib/python3.9/site-packages/seaborn/relational.py", line 485, in lineplot
  File "/Users/poset/Desktop/MIT/ml_approx/.venv/lib/python3.9/site-packages/seaborn/relational.py", line 216, in __init__
  File "/Users/poset/Desktop/MIT/ml_approx/.venv/lib/python3.9/site-packages/seaborn/_base.py", line 634, in __init__
  File "/Users/poset/Desktop/MIT/ml_approx/.venv/lib/python3.9/site-packages/seaborn/_base.py", line 679, in assign_variables
  File "/Users/poset/Desktop/MIT/ml_approx/.venv/lib/python3.9/site-packages/seaborn/_core/data.py", line 58, in __init__
  File "/Users/poset/Desktop/MIT/ml_approx/.venv/lib/python3.9/site-packages/seaborn/_core

In [62]:
# weight distribution

# alexnet
import torch
from alexnet_cifar10 import AlexNet
alexnet = AlexNet(num_classes=10)
alexnet.load_state_dict(torch.load("modelweights"))
weight_alexnet = alexnet.features[21].weight.data.flatten().numpy()

# distilbert
from transformers import AutoModelForSequenceClassification
distilbert = AutoModelForSequenceClassification.from_pretrained("textattack/distilbert-base-uncased-ag-news")
weight_distilbert = distilbert.pre_classifier.weight.data.flatten().numpy()

sns.histplot(weight_alexnet, bins=100, stat="density", label="AlexNet")
sns.histplot(weight_distilbert, bins=100, stat="density", label="DistilBERT")
plt.title("Weight Distributions")
pretty_graph()

ModuleNotFoundError: No module named 'torch'